属性查询的优先级
1. getattribute()， 无条件调用
2. 数据描述符：由1. 触发调用 （若人为的重载了该 getattribute() 方法，可能会导致无法调用描述符）
3. 实例对象的字典（若与描述符对象同名，会被覆盖哦）
4. 类的字典
5. 非数据描述符
6. 父类的字典
7. getattr() 方法

In [3]:
import pprint
pp = pprint.PrettyPrinter(indent=4, compact=False).pprint
import attrs
from attrs import define, field

In [4]:
import pprint
pp = pprint.PrettyPrinter(indent=4, compact=False).pprint
from attrs import define, field
                    
class Test(object):
    cls_val = 1
    def __init__(self):
        self.ins_val = 10

t = Test()
# pp.pprint(t.__dict__)
# pp.pprint(Test.__dict__)
pp(t.__dict__)
pp(Test.__dict__)

{'ins_val': 10}
mappingproxy({   '__dict__': <attribute '__dict__' of 'Test' objects>,
                 '__doc__': None,
                 '__init__': <function Test.__init__ at 0x000001D1969EE550>,
                 '__module__': '__main__',
                 '__weakref__': <attribute '__weakref__' of 'Test' objects>,
                 'cls_val': 1})


In [5]:
class Student:
    def __init__(self, name, math, chinese, english):
        self.name = name
        self.math = math
        self.chinese = chinese
        self.english = english
    @property
    def math(self):
        return self._math
    @math.setter
    def math(self, value):
        if 0 <= value <= 100:
            self._math = value
        else:
            raise ValueError('math score must be between 0 and 100')
    @property
    def chinese(self):
        return self._chinese
    @chinese.setter
    def chinese(self, value):
        if 0 <= value <= 100:
            self._chinese = value
        else:
            raise ValueError('chinese score must be between 0 and 100')
    @property
    def english(self):
        return self._english
    @english.setter
    def english(self, value):
        if 0 <= value <= 100:
            self._english = value
        else:
            raise ValueError('english score must be between 0 and 100')
    def __repr__(self):
        return f"Student: {self.name}, math: {self.math}, chinese: {self.chinese}, english: {self.english}"

stu = Student('Tom', 90, 80, 70)
pp(stu)
print(stu)

Student: Tom, math: 90, chinese: 80, english: 70
Student: Tom, math: 90, chinese: 80, english: 70


In [6]:
@define(slots=False)
class Score:
    def __init__(self, score=0, subject=None) -> None:
        self._score = score
        self._subject = subject
    def __get__(self, instance, owner):
        return self._scocre
    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise ValueError(f'{self._subject} score must be int')
        if not 0 <= value <= 100:
            raise ValueError(f'{self._subject} score must be between 0 and 100')
        self._scocre = value
    def __delete__(self):
        del self._scocre

@define(slots=False)
class Student:
    math = Score(score=0, subject='math')
    chinese = Score(score=0, subject='chinese')
    english = Score(score=0, subject='english')
    
    def __attrs_pre_init__(self):
        print('attrs_pre_init')


    """
    attrs: custom init
    
    """ 
    name: str
    math: int
    chinese: int
    english: int

    def __init__(self, name, math, chinese, english):
        self.__attrs_init__(name, math, chinese, english)
        print("custome init")

    """
    attrs: post_init
    """
    # name: str = field()
    # math: int = field()
    # chinese: int = field()
    # english: int = field()

    def __attrs_post_init__(self):
        print('attrs_post_init')
    # def __init__(self, name, math, chinese, english):
    #     self.name = name
    #     self.math = math
    #     self.chinese = chinese
    #     self.english = english

    def __repr__(self):
        return f"Student: {self.name}, math: {self.math}, chinese: {self.chinese}, english: {self.english}"

stu = Student('Tom', 90, 80, 70)
pp(attrs.asdict(stu))
pp(stu)
        

attrs_pre_init
attrs_post_init
custome init
{'chinese': 80, 'english': 70, 'math': 90, 'name': 'Tom'}
Student: Tom, math: 90, chinese: 80, english: 70


In [7]:
@define(slots=False)
class DataDes:
    _score: field(default=0)
    # def __init__(self, score=0):
    #     self._score = score
    def __set__(self, instance, value):
        self._score = value
    def __get__(self, instance, owner):
        print("访问数据描述符里的__get__")
        return self._score

@define(slots=False)
class NonDataDes:
    _score: field(default=0)
    # def __init__(self, score=0):
    #     self._score = score
    def __get__(self, instance, owner):
        print("访问非数据描述符里的__get__")
        return self._score


@define(slots=False)
class Student:
    _math = DataDes(0)
    _chinese = NonDataDes(0)

    _name: str
    _math: int
    _chinese: int
    
    def __getattribute__(self, item):
        print(f"调用__getattribute__{item}")
        return super().__getattribute__(item)
    
    def __repr__(self):
        return f"Student: {self._name}, math: {self._math}, chinese: {self._chinese}"
    
stu = Student(chinese = 90, math = 80, name='Tom')
pp(attrs.asdict(stu))

# pp(attrs.asdict(stu))
# pp(stu._math)
# pp(stu._chinese)
# print(stu)




访问数据描述符里的__get__
访问非数据描述符里的__get__
调用__getattribute____class__
调用__getattribute___name
调用__getattribute___math
调用__getattribute___chinese
{'_chinese': 90, '_math': 80, '_name': 'Tom'}


In [8]:
a = {
    "1": {
        "1": {
            "1": []}},
    "2": {
        "1": [],
        "2": {
            "1": []
        }
    }
}
def give_value(data):
    for k, v in data.items():
        # print(k, v)
        if isinstance(v, list):
            print("Yes")
            data[k] = 1
            # v = 1
        else:
            print(v)
            give_value(v)
give_value(a)
print(a)

{'1': {'1': []}}
{'1': []}
Yes
{'1': [], '2': {'1': []}}
Yes
{'1': []}
Yes
{'1': {'1': {'1': 1}}, '2': {'1': 1, '2': {'1': 1}}}


In [9]:
class Data:
    def __init__(self):
        pass


idx = 0
def create_node(data, value):
    data.__setattr__(str(value), value)

data = Data()
setattr(data, "1", Data())
setattr(getattr(data,"1"), "1", Data())

data.__setattr__("2", Data())
data.__getattribute__("2").__setattr__("3", 15)
 
print(data)
print(data.__dict__)
create_node(data.__getattribute__("2"), 15)

print(data.__getattribute__("2").__dict__)

{'1': <__main__.Data object at 0x000001D1969B0EB0>, '2': <__main__.Data object at 0x000001D196ABCD90>}
{'3': 15, '15': 15}


In [10]:
from dataclasses import dataclass
@dataclass
class Data:
    

SyntaxError: unexpected EOF while parsing (891763077.py, line 4)

In [ ]:
class Test:
    def __init__(self):
        pass
    def __getitem__(self, key):
        if isinstance(key, int):
            return self.List[key]
        elif isinstance(key, str):
            return self.__dict__[key]
    def __setitem__(self, key, value):
        if isinstance(key, str):
            if 'Dict' not in self.__dict__:
                self.__dict__['Dict'] = {}
            self.__dict__['Dict'][key] = value
        if isinstance(key, int):
            if 'List' not in self.__dict__:
                self.__dict__['List'] = [0, 0, 0]
            self.__dict__['List'][key] = value

t = Test()
t['a'] = 1
t[0] = 2

print(t.__dict__)




{'Dict': {'a': 1}, 'List': [2, 0, 0]}


In [ ]:

from dataclasses import dataclass
# @dataclass
class OD(dict):
    def __init__(self, *args, **kwargs):
        # print(self.__dict__)
        # print(dir(self))
        super().__init__(*args, **kwargs)
        # print(self.__dict__)
        # print(dir(self))
        # self.__dict__ = self
    # def __getattr__(self, name):
    #     value = self[name]
    #     print(value)
    #     if isinstance(value, dict):
    #         value = OD(value)
    #         print("    {0}", value)
    #     return value

od = OD(asf={'a': [1, 2, 3], 'b': 2}, b=2)
od.asf = OD(s=1)
# od.asf = OD(a=1, b=2)
# print(od.b)
print(od.asf)
print(od)
print(od.__dict__)
# print(od.__getattr__('a'))
# print(dict.__dict__)


{'s': 1}
{'asf': {'a': [1, 2, 3], 'b': 2}, 'b': 2}
{'asf': {'s': 1}}


In [ ]:

class OD(dict):
    def __init__(self, *args, **kwargs):
        pass

od = OD(asf=1, b=2)
print(od)

{}


In [ ]:
class Data:
    def __init__(self, t):
        if isinstance(t, str):
            if t in ('list', 'dict'):
                self._type = {'list': list, 'dict': dict}[t]
            else:
                raise ValueError("type string must be 'list' or 'dict'")
        elif isinstance(t, type):
            if t in (list, dict):
                self._type = t
            else:
                raise ValueError("type must be list or dict")
        else:
            raise ValueError("type must be string or type")
        
    def __setitem__(self, index, value):
        self.__dict__[str(index)] = value
        # self.__settattr__(str(index), value)
    def __getitem__(self, index):
        return self.__dict__[str(index)]
        # return self.__getattr__(str(index))
    def __setattr__(self, index, value):
        self.__dict__[str(index)] = value
    def __getattr__(self, index, value):
        return self.__dict__[str(index)]
    @property
    def value(self):
        keys = set(self.__dict__.keys())
        keys.remove('_type')
        if self._type == list:
            sorted_keys = sorted([int(i) for i in keys])
            rlt = [self.__dict__[str(i)] for i in sorted_keys]
            return rlt
        if self._type == dict:
            rlt = {i: self.__dict__[i] for i in keys}
            return rlt

            
        


data = Data(list)
data[1] = "abc"
data[2]= "def"
print(data.__dict__)
print(data.value)

data2 = Data(dict)
data2[1] = "qwe"
data2[2] = "asd"
print(data2.__dict__)
print(data2.value)
    
        
    

{'_type': <class 'list'>, '1': 'abc', '2': 'def'}
['abc', 'def']
{'_type': <class 'dict'>, '1': 'qwe', '2': 'asd'}
{'1': 'qwe', '2': 'asd'}


In [2]:
import xlwings as xw
import os
import re
wb_path = r"D:\GIT\Github\easyxl\xlsx\test.xlsx"
app = xw.App(visible=True, add_book=False)
wb = app.books.open(wb_path)
print(os.path.dirname(wb.fullname))
# ws_name = "cmd"
# class Range:
#     pass
# import os
# from pathlib import Path

# wb_path = Path(wb_path)
# output_path = wb_path / "output"#os.path.join(wb_path.parent, "output")


# print(output_path)

# print(wb_path.parent)

# app = xw.App(visible=True, add_book=False)
# with app.books.open(wb_path) as wb:
#     ws = wb.sheets[ws_name]
#     rng = ws.range((1, 1), (1, 2)).value
#     print(rng)







# app.quit()

D:\GIT\Github\easyxl\xlsx


In [ ]:
pr = True
def print_return(pr=True):
    def deco(func):
        def wrapper(*args, **kwargs):
            rlt = func(*args, **kwargs)
            if pr :
                print(rlt)
            return rlt
        return wrapper
    return deco
        

# @print_return(pr)
# def is_digit(s):
#     return s.isdigit()

import re
def col_to_num(col_str):
    """ Convert base26 column string to number. """
    expn = 0
    col_num = 0
    for char in reversed(col_str):
        col_num += (ord(char) - ord('A') + 1) * (26 ** expn)
        expn += 1
    return col_num

@print_return(pr)
def convert_address(address):
    reg = re.compile(r"([a-zA-Z]+|[0-9]+)")
    # reg_num = re.compile(r"[0-9]+")
    strs = reg.findall(address)
    print(strs)
    rlts = []
    for s in strs:
        if s.isdigit():
            rlts.append(int(s))
        else:
            rlts.append(col_to_num(s.upper()))
    if len(rlts) == 1:
        result = rlts[0]
    elif len(rlts) == 2:
        result = (rlts[0], rlts[1])
    elif len(rlts) ==4:
        result = ((rlts[0], rlts[1]), (rlts[2], rlts[3]))

    return result


a = "$CA$1:$J$1"
b = "ah"
c = "a12"
# print(convert_address(a))
# print(convert_address(b))
# print("12".isdigit())
# is_digit("12")
convert_address(a)
convert_address(b)
convert_address(c)




['CA', '1', 'J', '1']
((79, 1), (10, 1))
['ah']
34
['a', '12']
(1, 12)


(1, 12)

In [103]:
    
class Data:
    def __init__(self, t):
        if t in ('list', 'dict'):
            self._type = t
        else:
            print(t)
            raise ValueError("type input error")
    
    def __setitem__(self, index, value):
        self.__dict__[str(index)] = value
    def __getitem__(self, index):
        return self.__dict__[str(index)]
    def __setattr__(self, index, value):
        self.__dict__[str(index)] = value
    def __getattr__(self, index):
        return self.__dict__[str(index)]


def render_value(obj):
    if type(obj) is Data:
        keys = set(obj.__dict__.keys())
        keys.remove('_type')
        # print(keys)
        if obj._type == list:
            sorted_keys = sorted([int(i) for i in keys])
            for i in sorted_keys:
                # print("{0} {1}".format(obj.__dict__, type(obj.__dict__[str(i)])))
                if type(obj.__dict__[str(i)]) is Data:
                    rlt = [render_value(obj.__dict__[str(i)]) for i in sorted_keys]
                else:
                    rlt = [obj[str(i)] for i in sorted_keys]
        if obj._type == dict:
            for i in keys:
                # print("{0} {1}".format(obj.__dict__, type(obj.__dict__[i])))
                if type(obj.__dict__[i]) is Data:
                    rlt = {i: render_value(obj.__dict__[i]) for i in keys}
                else:
                    rlt = {i: obj[i] for i in keys}
    else:
        rlt = obj
    return rlt

data = Data('dict')
data.a = 1
data.b = Data('list')
data.c = "abcd"
data.b[1] = Data('dict')
data.b[1].a = 1
data.b[0] = Data('dict')
data.b[0].a = 2

# print(data)
r = render_value(data)
print(r)

UnboundLocalError: local variable 'rlt' referenced before assignment

In [7]:
def render_value(obj):
    keys = set(obj.__dict__.keys())
    keys.remove('_type')
    if obj._type == 'list':
        sorted_keys = sorted([int(i) for i in keys])
        l = len(sorted_keys)
        remap_keys = range(l)
        m = {remap_keys[i]:str(sorted_keys[i]) for i in range(l)}
        rlt = [None for i in range(l)]
        for idx, attr in m.items():
            if type(obj.__dict__[attr]) is Data:
                rlt[idx] = render_value(obj.__dict__[attr])
            else:
                rlt[idx] = obj[attr]
    if obj._type == 'dict':
        rlt = {}
        for i in keys:
            if type(obj.__dict__[i]) is Data:
                rlt[i] = render_value(obj.__dict__[i])
            else:
                rlt[i] = obj[i]
    return rlt

class Data:
    def __init__(self, t):
        if t in ('list', 'dict'):
            self._type = t
        else:
            print(t)
            raise ValueError("type input error")
    
    def __setitem__(self, index, value):
        self.__dict__[str(index)] = value
    def __getitem__(self, index):
        return self.__dict__[str(index)]
    def __setattr__(self, index, value):
        self.__dict__[str(index)] = value
    def __getattr__(self, index):
        return self.__dict__[str(index)]


data = Data('dict')
data.a = 1
data.b = Data('list')
data.c = "abcd"
data.b[1] = Data('dict')
data.b[1].a = Data('list')
data.b[1].a[0] = "here"
data.b[1].a[1] = Data('dict')
data.b[1].a[1]["maybe"] = "maybe"


data.b[0] = Data('dict')
data.b[0].a = [1, 2,3, ]

# print(data)
r = render_value(data)
print(r)

{'a': 1, 'b': [{'a': [1, 2, 3]}, {'a': ['here', {'maybe': 'maybe'}]}], 'c': 'abcd'}


In [ ]:
class Node:
    def __init__(self, value) -> None:
        self._value = value
        self._children = []
    def __repr__(self) -> str:
        return f"Node({self._value})"
    def add_child(self, node):
        self._children.append(node)
    def __iter__(self):
        return iter(self._children)
    def depth_first(self):
        yield self
        for c in self:
            yield from c.depth_first()

root = Node(0)
c1 = Node(1)
c2 = Node(2)
root.add_child(c1)
root.add_child(c2)
c1.add_child(Node(3))
c2.add_child(Node(4))

for i in root.depth_first():
    print(i)


Node(0)
Node(1)
Node(3)
Node(2)
Node(4)


In [18]:
a = (1, 2)
c = [[1, 2]]
b = tuple([a])
c = tuple(c)
print(len(b))
print(c)


1
([1, 2],)


In [ ]:

class Countdown:
    def __init__(self, step) -> None:
        print("start init")
        self.step = step
        print("init done\n")
    def __next__(self):
        print("start next with step: {0}".format(self.step))
        if self.step > 5:
            raise StopIteration
        self.step += 1
        print("end next with step: {0}\n".format(self.step))
        return self
    def __iter__(self):
        return self#.__next__()

# c = Countdown(0)
# next(c)
# for i in c:
#     print(i)
# for idx, i in enumerate(c):
#     print(idx, i.step)

# for i in range(12):
#     # print(c.step)
#     print(i)
#     print("step{0}".format(c.step))
#     next(c)

for idx, i in enumerate(Countdown(0)):
    print("idx {0}".format(idx))
    # print(i.step)
# print(list(Countdown(0)))




In [ ]:
def func():
    print("start func")
    while True:
        res = yield  4
        print("res: {0}".format(res))


f = func()
print("###")
print(next(f))
print("===")
print(next(f))
print("+++")
print(next(f))
print("---")
print(next(f))

In [86]:
class C:
    def __init__(self, step=0):
        self.step = step
        print("init called\n")
        pass
    def __iter__(self):
        print("iter called\n")
        return self
    def __next__(self):
        print("next called")
        print("self.step before += 1: {0}".format(self.step))
        if self.step > 5:
            raise StopIteration
        self.step += 1
        print("self.step after += 1: {0}\n".format(self.step))
        # yield self
        return self
        print('yield done')

c = C()
print("===")
for idx, i in enumerate(c):
    print(idx)
    print(i.step)
    # next(i)
    # print(i.step)



init called

===
iter called

next called
self.step before += 1: 0
self.step after += 1: 1

0
1
next called
self.step before += 1: 1
self.step after += 1: 2

1
2
next called
self.step before += 1: 2
self.step after += 1: 3

2
3
next called
self.step before += 1: 3
self.step after += 1: 4

3
4
next called
self.step before += 1: 4
self.step after += 1: 5

4
5
next called
self.step before += 1: 5
self.step after += 1: 6

5
6
next called
self.step before += 1: 6


In [112]:
from io import StringIO
output = StringIO()
output.write("abcdefg")
output.seek(2)
print(output.getvalue())
print(output.tell())
# print(next(output))

abcdefg
2


In [120]:
j_path = r"G:\Seafile\工程仓库\triple_match_3d\extracted_files\TextAsset\FinalLoopLevelsGroup.json"
import json5
with open(j_path, 'r', encoding='utf-8') as f:
    j = json5.load(f)
    all_min = {}
    all_max = {}
    lst_100_2010 = set(range(100, 2011))
    not_selected = []
    j = sorted(j.items(), key=lambda x: int(x[0]))
    for k, v in j:
        print("{0}: {1}".format(k, len(v)))
        all_min[k] = min(v)
        all_max[k] = max(v)
        v_set = set(v)
        lst_100_2010 = lst_100_2010 - v_set
print(sorted(list(lst_100_2010)))
print(all_min)
print(all_max)
print(min(all_min.values()))
print(max(all_max.values()))

0: 1166
1: 370
2: 166
3: 24
4: 119
5: 4
8: 3
9: 59
[]
{'0': 101, '1': 105, '2': 119, '3': 111, '4': 167, '5': 145, '8': 100, '9': 112}
{'0': 2010, '1': 2003, '2': 2004, '3': 1866, '4': 2000, '5': 1667, '8': 266, '9': 1996}
100
2010


In [6]:
content_row = 3
a = ((1, 2), (1, 5))
b = ((1, 2), )
a_ = tuple([(content_row, a[i][1]) for i in range(len(a))])
b_ = tuple([(content_row, b[i][1]) for i in range(len(b))])
print(a_)
print(b_)
import xlwings as xw
wb_path = r"D:\GIT\Github\TinyTool\xlsx\test.xlsx"
wb = xw.Book(wb_path)
ws = wb.sheets("Test2")
value = ws.range(*b_).value
print(value)

((3, 2), (3, 5))
((3, 2),)
beginner_bundle


In [16]:
import json5
import re
a = "[1, 2, 3, 4, 5]"
b = '{"fasd": 12, "e": [12, 342]}'
print(json5.loads(a))
json_str_reg = re.compile(r"(^\[[\s\S]*\]$)|(^\{[\s\S]*\}$)")
json_str_reg = re.compile(r"^\[[\s\S]*\]$|^\{[\s\S]*\}$")
print(json_str_reg.findall(a))
print(json_str_reg.match(a))
print(json_str_reg.findall(b))
print(json_str_reg.match(b))

[1, 2, 3, 4, 5]
['[1, 2, 3, 4, 5]']
<re.Match object; span=(0, 15), match='[1, 2, 3, 4, 5]'>
['{"fasd": 12, "e": [12, 342]}']
<re.Match object; span=(0, 28), match='{"fasd": 12, "e": [12, 342]}'>


In [5]:
import re
a = "%123%4"
s = re.split(r"%", a)
print(s)

['', '123', '4']


In [13]:
li = [1, 2, 3, 4, 5, 4, 3, 1, 2]
a = next(i for i in reversed(range(len(li))) if li[i] == 1)
print(a)
# print(a.next())
b = iter(li)
print(next(b))
print(next(b))
e = iter(i for i in reversed(range(len(li))) if li[i] == 1)
print(next(e))
print(next(e))



7
1
2
7
0


In [28]:
a = [1, 2, 3, 4, 5]
b = ((1, 2), (1, 4))
c = ((1, 2),)
d = (((1, 2),))
print(*a)
print(*b)
print(*c)
print(*d)
def range_value_from_list(li, rng):
    if len(rng) == 1:
        return li[rng[0][1]-1]
    elif len(rng) == 2:
        return li[rng[0][1]-1:rng[1][1]] 
print(range_value_from_list(a, b))
print(range_value_from_list(a, c))

1 2 3 4 5
(1, 2) (1, 4)
(1, 2)
(1, 2)
[2, 3, 4]
2


In [37]:
class A:
    e = 1
    def __init__(self):
        self.a = 1
    class B:
        c = 2
        def __init__(self):
            self.b = 2

ca = A()
cb = ca.B()
print(cb.c)

2


In [6]:
j_path = r"D:\GIT\Github\TinyTool\xlsx\output\Test1.json"
import json5
with open(j_path, 'r', encoding='utf-8') as f:
    j = json5.load(f)
    print(j)

{'bundle1': [{'DA': '3.0', 'LA': [1, 2], 'LB': [[110, 120]], 'LC': [[1.0, 14.0, {'Lcab': 14.0}, {'LCac': 15.0}], [16.0, 17.0, 18.0], [{'LCCaa': 19.0, 'LCCab': 20.0, 'LCCac': 21.0}, [{'LCCba': None}, {'LCCbc': None}, {'LCCbd': None}]]], 'LD': [{'LDa': 22.0}, {'LDc': 25.0}, {'LDe': 26.0}, {'LDf': 27.0}, {'LDg': 28.0}, {'LDh': 29.0}]}, {'DA': '8.0', 'LA': [6, 7], 'LB': [{'a': 1, 'b': 2}], 'LC': [[2.0, 3.0, {'Lcab': 19.0}, {'LCac': 20.0}], [21.0, 22.0, 23.0], [{'LCCaa': 24.0, 'LCCab': 25.0, 'LCCac': 26.0}, [{'LCCba': 5.0}, {'LCCbc': 5.0}, {'LCCbd': 5.0}]]], 'LD': [{'LDa': 27.0}, {'LDc': 30.0}, {'LDe': 31.0}, {'LDf': 32.0}, {'LDg': 33.0}, {'LDh': 34.0}]}], 'bundle3': {'DA': '13.0', 'LA': [11, 12], 'LB': [15], 'LC': [[7.0, 8.0, {'Lcab': 24.0}, {'LCac': 25.0}], [26.0, 27.0, 28.0], [{'LCCaa': 29.0, 'LCCab': 30.0, 'LCCac': 31.0}, [{'LCCba': 10.0}, {'LCCbc': 10.0}, {'LCCbd': 10.0}]]], 'LD': [{'LDa': 32.0}, {'LDc': 35.0}, {'LDe': 36.0}, {'LDf': 37.0}, {'LDg': 38.0}, {'LDh': 39.0}]}, 'None': None}

In [12]:
import re
header_split_char = "-"
null_tags = ['null', 'Null', 'NULL', 'none', 'None', 'NONE']
def first_n_last_con_occ(in_list, check_str):
    first_occ = None
    last_occ = None
    for idx, item in enumerate(in_list):
        if first_occ is None:
            if check_str in split_text(item):
                if not str(item).startswith("#"):
                    first_occ = idx + 1
                else:
                    continue
            else:
                continue
        else:
            if check_str in split_text(item):
                if not str(item).startswith("#"):
                    last_occ = idx + 1
                else:
                    continue
            else:
                pass
    if last_occ is None:
        last_occ = first_occ
    return first_occ, last_occ

                
def split_text(input):
    input = str(input)
    output = [i.strip() for i in re.split(header_split_char, input)]
    return output

def check_tags_in_src(src, tags):
    b = False
    src = split_text(src)
    for tag in tags:
        if tag in src:
            b = True
            break
    return b
a = "-none-int-list"
b = ["abc-int-h", "#id5-h", "#id6-h", "id7-h", "#cd-h"]#, "id8", "id9"]
print(check_tags_in_src(a, null_tags))
print(first_n_last_con_occ(b, "h"))

True
(1, 4)


In [5]:
import json5
j = r"C:\Users\HPC\Documents\vscode_settings.json"
out = r"C:\Users\HPC\Documents\out.json"
with open(j, 'r', encoding='utf-8') as f:
    j = json5.load(f)
with open(out, 'w', encoding='utf-8') as f:
    f.write(json5.dumps(j, indent=4, ensure_ascii=False, \
        sort_keys=True, quote_keys=True, trailing_commas=False))
    # print(j)

In [19]:
import time
def time_deco(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        func(*args, **kwargs)
        end = time.time()
        print("time: {}".format(end-start))
    return wrapper

a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# b = set(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
a = None
b = "#None"

@time_deco
def func(src):
    bool_ = True
    for i in range(1000000):
        if type(a) is str and a.startwith("#"):
            pass

@time_deco
def func2(src):
    bool_ = True
    for i in range(1000000):
        if str(src).startswith("#"):
            pass
@time_deco
def func3(src):
    bool_ = True
    for i in range(1000000):
        sr = set(src)
        if 10 in sr:
            bool_ = True
    
    

func(a)
func2(a)
func(b)
func2(b)

# func3(a)
            
        

time: 0.05505037307739258
time: 0.14513158798217773
time: 0.056050777435302734
time: 0.12311196327209473


In [5]:
a = [1, 1]
b = [1, 1]


[('001', '2', '3'), ('004', '5', '6'), ('a', 'b', 'c'), ('d', 'e', 'f')]
00001


In [18]:
a = [0, -1, 1.5, 1, "0", "1", None, "", " ", {}, [], set()]
# for i in a:
#     print(bool(i))
c = "1.0"
e = "e.1"
print(c.isdigit())
import re
number_reg = re.compile(r"^[0-9\.]+$")
print(number_reg.match(c))
print(number_reg.match(e))
def convert_name(instr):
    rlt = instr
    if number_reg.match(instr):
        rlt = str(int(float(instr)))
    return rlt

print(convert_name(c))

False
<re.Match object; span=(0, 3), match='1.0'>
None


ValueError: invalid literal for int() with base 10: '1.0'

In [55]:
class UnifiedData:
    # 标记数据类型，把数据注册到这个类里面
    def __init__(self, t):
        if t in ('list', 'dict'):
            self._type = t
        else:
            # print(t)
            raise ValueError("type input error {0}".format(t))
    
    def __setitem__(self, index, value):
        self.__dict__[str(index)] = value
    def __getitem__(self, index):
        return self.__dict__[str(index)]
    def __setattr__(self, index, value):
        self.__dict__[str(index)] = value
    def __getattr__(self, index):
        return self.__dict__[str(index)]

def render_unified_data(obj, ignore_null=True):
    # print("render_unified_data is called")
    keys = set(obj.__dict__.keys())
    for i in ['_type', '_has_content']:
        if i in keys:
            keys.remove(i)
    # 如果数据类型是list，那么需要对key进行排序，按照数字大小排序输出为list
    if obj._type == 'list':
        # if ignore_null:
        #     sorted_keys = sorted([int(i) for i in keys if obj.__dict__[i] is not None])
        #     # print("ignore is True and sorted_keys is {0}".format(sorted_keys))
        # else:
        sorted_keys = sorted([int(i) for i in keys])
            # print("ignore is False and sorted_keys is {0}".format(sorted_keys))
        l = len(sorted_keys)
        remap_keys = range(l)
        m = {remap_keys[i]:str(sorted_keys[i]) for i in range(l)}
        rlt = [None for i in range(l)]
        for idx, attr in m.items():
            if type(obj.__dict__[attr]) is UnifiedData:
                rlt[idx] = render_unified_data(obj.__dict__[attr])
            else:
                rlt[idx] = obj[attr]
    # 如果数据类型是dict，那么直接输出为dict
    elif obj._type == 'dict':
        rlt = {}
        for i in keys:
            # print(obj.__dict__[i])
            if type(obj.__dict__[i]) is UnifiedData:
                rlt[i] = render_unified_data(obj.__dict__[i])
            else:
                # print(obj.__dict__[i])
                # if ignore_null and obj.__dict__[i] is None:
                #     print("found empty cell")
                #     continue
                # else:
                rlt[i] = obj[i]
    else:
        raise ValueError("obj._type error")
    return rlt

def tag_data(obj):
    keys = set(obj.__dict__.keys())
    for i in ['_type', '_has_content']:
        if i in keys:
            keys.remove(i)
    has_content = []
    for key in keys:
        if type(obj[key]) is UnifiedData:
            has_content.append(tag_data(obj[key]))
        else:
            if obj[key]:
                has_content.append(1)
            else:
                has_content.append(0)
    rlt = 1 if sum(has_content) else 0
    obj._has_content = rlt
    return rlt

def prune_data(obj):
    keys = set(obj.__dict__.keys())
    for i in ['_type', '_has_content']:
        if i in keys:
            keys.remove(i)
    for key in keys:
        if type(obj[key]) is UnifiedData:
            if obj[key]._has_content == 0:
                print("del key: {0}".format(key))
                obj.__delattr__(key)
            else:
                prune_data(obj[key])
                

    
data = UnifiedData('dict')
data.a = 1
data.b = 2
data.c = UnifiedData('list')
data.c[0] = UnifiedData('dict')
data.c[1] = UnifiedData('list')
data.c[0].a = 1


# print(data.__dict__)
# print(render_unified_data(data))

tag_data(data)
prune_data(data)

# print(data.c.__dict__)
# print(data.c._has_content)
print(render_unified_data(data))

# print(sum([]))

# print(render_unified_data(d2))


del key: 1
{'a': 1, 'c': [{'a': 1}], 'b': 2}


In [4]:
import json
from typing import List

def generate_class(class_name: str, data: dict) -> str:
    class_template = f"class {class_name} {{\n"
    for key, value in data.items():
        if isinstance(value, dict):
            class_template += f"    public {key.capitalize()}: {generate_class(key.capitalize(), value)};\n"
        elif isinstance(value, list):
            if isinstance(value[0], dict):
                class_template += f"    public List<{key.capitalize()}> {key.capitalize()}: {generate_class(key.capitalize(), value[0])}[];\n"
            else:
                class_template += f"    public List<{type(value[0]).__name__}> {key.capitalize()};\n"
        else:
            class_template += f"    public {key.capitalize()}: {type(value).__name__};\n"
    class_template += "}"
    return class_template

if __name__ == "__main__":
    json_data = '{"name": "John", "age": 30, "city": "New York", "pets": [{"name": {"dog": "korgi"}, "type": "cat"}, {"name": "Fido", "type": "dog"}]}'
    data = json.loads(json_data)
    class_name = "Person"
    class_template = generate_class(class_name, data)
    print(class_template)


class Person {
    public Name: str;
    public Age: int;
    public City: str;
    public List<Pets> Pets: class Pets {
    public Name: class Name {
    public Dog: str;
};
    public Type: str;
}[];
}


In [2]:
import json
from typing import List

def json_to_csharp_class(json_data, class_name):
    csharp_class = f"public class {class_name}\n{{\n"
    for key, value in json_data.items():
        if isinstance(value, dict):
            nested_class_name = key.capitalize()
            nested_class = json_to_csharp_class(value, nested_class_name)
            csharp_class += f"    public {nested_class_name} {key.capitalize()} {{ get; set; }} = new {nested_class_name}();\n"
            csharp_class += nested_class
        else:
            csharp_type = "string"
            if isinstance(value, bool):
                csharp_type = "bool"
            elif isinstance(value, int):
                csharp_type = "int"
            elif isinstance(value, float):
                csharp_type = "double"
            csharp_class += f"    public {csharp_type} {key.capitalize()} {{ get; set; }} = {value};\n"
    csharp_class += "}\n\n"
    return csharp_class

if __name__ == "__main__":
    json_data = '{"name": "John", "age": 30, "city": "New York", "pets": [{"name": "Fluffy", "type": "cat"}, {"name": "Fido", "type": "dog"}]}'
    data = json.loads(json_data)
    class_name = "Person"
    class_template = json_to_csharp_class(json_data, class_name)
    print(class_template)


AttributeError: 'str' object has no attribute 'items'

In [5]:
a = list()
b = dict()
c = tuple()
e = set()
f = [list(), dict(), tuple(), set()]
print(a in f)
print(b in f)
print(c in f)
print(e in f)

True
True
True
True
